# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fafc0352250>
├── 'a' --> tensor([[ 0.6133,  0.2586,  2.7233],
│                   [-0.3295,  0.3413,  0.3055]])
└── 'x' --> <FastTreeValue 0x7fafc0352160>
    └── 'c' --> tensor([[ 0.6550,  0.0210,  1.1992,  2.8888],
                        [-1.0356,  1.2582,  0.4922,  0.3346],
                        [-0.5677, -0.2699,  0.0894, -0.2671]])

In [4]:
t.a

tensor([[ 0.6133,  0.2586,  2.7233],
        [-0.3295,  0.3413,  0.3055]])

In [5]:
%timeit t.a

61.3 ns ± 0.0454 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fafc0352250>
├── 'a' --> tensor([[ 0.2037, -0.5824,  1.1687],
│                   [-0.1017,  1.5422,  0.4434]])
└── 'x' --> <FastTreeValue 0x7fafc0352160>
    └── 'c' --> tensor([[ 0.6550,  0.0210,  1.1992,  2.8888],
                        [-1.0356,  1.2582,  0.4922,  0.3346],
                        [-0.5677, -0.2699,  0.0894, -0.2671]])

In [7]:
%timeit t.a = new_value

67.3 ns ± 0.1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6133,  0.2586,  2.7233],
               [-0.3295,  0.3413,  0.3055]]),
    x: Batch(
           c: tensor([[ 0.6550,  0.0210,  1.1992,  2.8888],
                      [-1.0356,  1.2582,  0.4922,  0.3346],
                      [-0.5677, -0.2699,  0.0894, -0.2671]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6133,  0.2586,  2.7233],
        [-0.3295,  0.3413,  0.3055]])

In [11]:
%timeit b.a

60.3 ns ± 0.0295 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[0.0857, 0.8839, 0.1599],
               [0.5864, 1.0001, 1.9514]]),
    x: Batch(
           c: tensor([[ 0.6550,  0.0210,  1.1992,  2.8888],
                      [-1.0356,  1.2582,  0.4922,  0.3346],
                      [-0.5677, -0.2699,  0.0894, -0.2671]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.224 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

850 ns ± 0.261 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 35.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 452 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fafc0356c40>
├── 'a' --> tensor([[[ 0.6133,  0.2586,  2.7233],
│                    [-0.3295,  0.3413,  0.3055]],
│           
│                   [[ 0.6133,  0.2586,  2.7233],
│                    [-0.3295,  0.3413,  0.3055]],
│           
│                   [[ 0.6133,  0.2586,  2.7233],
│                    [-0.3295,  0.3413,  0.3055]],
│           
│                   [[ 0.6133,  0.2586,  2.7233],
│                    [-0.3295,  0.3413,  0.3055]],
│           
│                   [[ 0.6133,  0.2586,  2.7233],
│                    [-0.3295,  0.3413,  0.3055]],
│           
│                   [[ 0.6133,  0.2586,  2.7233],
│                    [-0.3295,  0.3413,  0.3055]],
│           
│                   [[ 0.6133,  0.2586,  2.7233],
│                    [-0.3295,  0.3413,  0.3055]],
│           
│                   [[ 0.6133,  0.2586,  2.7233],
│                    [-0.3295,  0.3413,  0.3055]]])
└── 'x' --> <FastTreeValue 0x7faf0b694760>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 73.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7faf12280e50>
├── 'a' --> tensor([[ 0.6133,  0.2586,  2.7233],
│                   [-0.3295,  0.3413,  0.3055],
│                   [ 0.6133,  0.2586,  2.7233],
│                   [-0.3295,  0.3413,  0.3055],
│                   [ 0.6133,  0.2586,  2.7233],
│                   [-0.3295,  0.3413,  0.3055],
│                   [ 0.6133,  0.2586,  2.7233],
│                   [-0.3295,  0.3413,  0.3055],
│                   [ 0.6133,  0.2586,  2.7233],
│                   [-0.3295,  0.3413,  0.3055],
│                   [ 0.6133,  0.2586,  2.7233],
│                   [-0.3295,  0.3413,  0.3055],
│                   [ 0.6133,  0.2586,  2.7233],
│                   [-0.3295,  0.3413,  0.3055],
│                   [ 0.6133,  0.2586,  2.7233],
│                   [-0.3295,  0.3413,  0.3055]])
└── 'x' --> <FastTreeValue 0x7faf0b63ef40>
    └── 'c' --> tensor([[ 0.6550,  0.0210,  1.1992,  2.8888],
                        [-1.0356,  1.2582,  0.4922,  0.3346],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 71.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.8 µs ± 69.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.6550,  0.0210,  1.1992,  2.8888],
                       [-1.0356,  1.2582,  0.4922,  0.3346],
                       [-0.5677, -0.2699,  0.0894, -0.2671]],
              
                      [[ 0.6550,  0.0210,  1.1992,  2.8888],
                       [-1.0356,  1.2582,  0.4922,  0.3346],
                       [-0.5677, -0.2699,  0.0894, -0.2671]],
              
                      [[ 0.6550,  0.0210,  1.1992,  2.8888],
                       [-1.0356,  1.2582,  0.4922,  0.3346],
                       [-0.5677, -0.2699,  0.0894, -0.2671]],
              
                      [[ 0.6550,  0.0210,  1.1992,  2.8888],
                       [-1.0356,  1.2582,  0.4922,  0.3346],
                       [-0.5677, -0.2699,  0.0894, -0.2671]],
              
                      [[ 0.6550,  0.0210,  1.1992,  2.8888],
                       [-1.0356,  1.2582,  0.4922,  0.3346],
                       [-0.5677, -0.2699,  0.0894, -0.2671]],

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.6550,  0.0210,  1.1992,  2.8888],
                      [-1.0356,  1.2582,  0.4922,  0.3346],
                      [-0.5677, -0.2699,  0.0894, -0.2671],
                      [ 0.6550,  0.0210,  1.1992,  2.8888],
                      [-1.0356,  1.2582,  0.4922,  0.3346],
                      [-0.5677, -0.2699,  0.0894, -0.2671],
                      [ 0.6550,  0.0210,  1.1992,  2.8888],
                      [-1.0356,  1.2582,  0.4922,  0.3346],
                      [-0.5677, -0.2699,  0.0894, -0.2671],
                      [ 0.6550,  0.0210,  1.1992,  2.8888],
                      [-1.0356,  1.2582,  0.4922,  0.3346],
                      [-0.5677, -0.2699,  0.0894, -0.2671],
                      [ 0.6550,  0.0210,  1.1992,  2.8888],
                      [-1.0356,  1.2582,  0.4922,  0.3346],
                      [-0.5677, -0.2699,  0.0894, -0.2671],
                      [ 0.6550,  0.0210,  1.1992,  2.8888],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 390 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 3.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
